In [1]:
%load_ext autoreload
%autoreload 2
# then imported files are u pdated each time a cell is executed (good if functions in py files are added and modified)

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import sys
import os
import pickle

from scipy.spatial.transform import Rotation as R

# os.add_dll_directory("C:/OpenSim4.4/bin") # otherwise module _sombody not found error!

sys.path.insert(0, '..')
sys.path.insert(0, '...')

from Python.LMC_util import plotLMC, make_LMC, check_LMC_Hand_visibility, make_hand_poses
from Python.plotUtil import plot_hand
from Python.LMC_util import fingers_idx_LMC

# Load and process data

In [ ]:
# General Settings
# Data Path
current_dir = os.getcwd()
data_path = 'data/LMC_raw'
save_path = 'data/LMC_rayTracing'

# LMC Settings
# Input parameter
finger_radius = 10 # mm

# LMC settings
num_LMC = 1
LMC_H = 400 # mm
LMC_alpha1 = (180-150) / 2 * np.pi / 180
LMC_alpha2 = (180-120) / 2 * np.pi / 180
LMC_position = np.array([0,0,0]) # in mm
LMC_orientation = np.array([0,0,0]) # in deg
LMC = make_LMC(LMC_loc=LMC_position, LMC_orient=LMC_orientation, LMC_H=LMC_H, LMC_alpha1=LMC_alpha1, LMC_alpha2=LMC_alpha2) 

# PROCES DATA
for subject in os.listdir(os.path.join(os.getcwd(),  'data', 'LMC_raw')):
    if os.path.isdir(os.path.join(os.getcwd(),  'data', 'LMC_raw', subject)):
        print('_________________________________________________________________________')
        print(subject)
        for trialtype in ['Naiv', 'Opti']:
            print('    ', trialtype)
            data_path = os.path.join(os.getcwd(),  'data', 'LMC_raw', subject, trialtype)
            save_path = os.path.join(os.getcwd(),  'data', 'LMC_rayTracing', subject, trialtype)
            # load files
            for fi, file in enumerate(os.listdir(os.path.join(current_dir, data_path))):
                if '_Trial' in file or '_Static' in file:
                    print('        ', file)
                    # load trial
                    filename = os.path.join(current_dir, data_path, file)
                    df = pd.read_csv(filename,index_col=False,sep=r',|\t', engine='python')
                    df = df.replace('\t','', regex=True)
                    #print(df.iloc[[0]].keys())
                    # to numpy
                    posDatas_ = df.to_numpy()
                    posDatas = posDatas_[:, [4, 5, 6] + [13+i for i in range(94-13)]] # cut away unnecessary data
                    posDatas = posDatas.reshape(posDatas.shape[0], int(posDatas.shape[1]/3), 3)
                    posDatas = posDatas[:, [i for i in range(posDatas.shape[1]) if i not in [0, 2, 3, 8, 13, 18]]] # cut away unnecessary markers

                    #print(posDatas.shape)
                    # check if labels are correct
                    #f = 11
                    # frame = (R.from_euler('XYZ', [90, 0, 0], degrees=True).as_matrix().squeeze() @ posDatas[f].T).T
                    # print(frame)
                    # fig = go.Figure()
                    # plot_hand(fig, frame, names=np.arange(0, frame.shape[0]))
                    # fig.show()

                    occlusions = np.zeros([posDatas.shape[0], 5])
                    for f in range(posDatas.shape[0]):  #for each frame
                        # rotate data such that z points up
                        frame = (R.from_euler('XYZ', [90, 0, 0], degrees=True).as_matrix().squeeze() @ posDatas[f].T).T
                        frame = frame.astype(np.float64) # otherwise there is a type error for some files
                        # transform data in global frame not necessary as we only have one LMC
                        if pd.isnull(frame).any(): # np.isnan(frame).any gives an error for the 'PRAM' subject the LMC did not detect the hand:
                            occlusions[f, :] = np.array([-1]*5)
                            continue
                        
                        # get data for ray tracing
                        handMarkers, palmMarkers, palmPlnNormals, palmCentroids, fingers, forearm_vecs = make_hand_poses(np.array([frame]))   
                        occlusions_, _ = check_LMC_Hand_visibility(LMC, pos_markers=handMarkers[0], fingers_idx=fingers_idx_LMC, finger_lines=fingers[0], 
                                                                    finger_radius=finger_radius, palm_plane_normal=palmPlnNormals[0], 
                                                                    palm_centroid=palmCentroids[0], palm_markers=palmMarkers[0], forearm_vec=forearm_vecs[0],
                                                                    fig = None, verbose=0)
                        occlusions[f, :] = occlusions_[2:]
                        #print('    Hand detected: {}'.format(f))
                    
                    fingers = ['thumb', 'index', 'middle', 'ring', 'pinky']
                    for finger in range(5):
                        df['visibility_'+fingers[finger]] = occlusions[:, finger]
                    savefilename = os.path.join(current_dir, save_path, file)
                    df.to_csv(savefilename, header=True, index=None, sep=' ', mode='w', quoting=1, quotechar=' ')
                    savefilename = os.path.join(current_dir, save_path, file[:-4] + '.pkl')
                    df.to_pickle(savefilename) #long therm storage

# Load all pickle files

In [6]:
# General Settings
# Data Path
current_dir = os.getcwd()
datapath = 'data/LMC_rayTracing'

# PROCES DATA
for subject in os.listdir(os.path.join(os.getcwd(),  datapath)):
    if os.path.isdir(os.path.join(os.getcwd(),  datapath, subject)):
        print('_________________________________________________________________________')
        print(subject)
        for trialtype in ['Naiv', 'Opti']:
            print('    ', trialtype)
            datapath_ = os.path.join(os.getcwd(),  'data', 'LMC_rayTracing', subject, trialtype)
            # load files
            for fi, file in enumerate(os.listdir(datapath_)):
                if '.pkl' in file:
                    print('        ', file)
                    df = pd.read_pickle(os.path.join(datapath_, file))
                    

_________________________________________________________________________
Chris
     Naiv
         LMC_Naiv_Static101.pkl
         LMC_Naiv_Static201.pkl
         LMC_Naiv_Trial101.pkl
         LMC_Naiv_Trial102.pkl
         LMC_Naiv_Trial103.pkl
         LMC_Naiv_Trial104.pkl
         LMC_Naiv_Trial105.pkl
         LMC_Naiv_Trial106.pkl
         LMC_Naiv_Trial107.pkl
         LMC_Naiv_Trial108.pkl
         LMC_Naiv_Trial109.pkl
         LMC_Naiv_Trial110.pkl
         LMC_Naiv_Trial111.pkl
         LMC_Naiv_Trial112.pkl
         LMC_Naiv_Trial113.pkl
         LMC_Naiv_Trial114.pkl
         LMC_Naiv_Trial115.pkl
         LMC_Naiv_Trial201.pkl
         LMC_Naiv_Trial202.pkl
         LMC_Naiv_Trial203.pkl
         LMC_Naiv_Trial204.pkl
         LMC_Naiv_Trial205.pkl
         LMC_Naiv_Trial206.pkl
         LMC_Naiv_Trial207.pkl
         LMC_Naiv_Trial208.pkl
         LMC_Naiv_Trial209.pkl
         LMC_Naiv_Trial210.pkl
         LMC_Naiv_Trial211.pkl
         LMC_Naiv_Trial212.pkl
         

# Debugging

In [ ]:
import warnings
warnings.simplefilter('error')

# LMC Settings
# Input parameter
finger_radius = 10 # mm

# LMC settings
num_LMC = 1
LMC_H = 400 # mm
LMC_alpha1 = (180-150) / 2 * np.pi / 180
LMC_alpha2 = (180-120) / 2 * np.pi / 180
LMC_position = np.array([0,0,0]) # in mm
LMC_orientation = np.array([0,0,0]) # in deg
LMC = make_LMC(LMC_loc=LMC_position, LMC_orient=LMC_orientation, LMC_H=LMC_H, LMC_alpha1=LMC_alpha1, LMC_alpha2=LMC_alpha2) 

current_dir = os.getcwd()
trialtype = 'Naiv'
#trialtype = 'Opti'
subject = 'Fajar2'
#subject = 'Pram'
data_path = os.path.join(os.getcwd(),  'data', 'LMC_raw', subject, trialtype)
file = 'Naiv_Trial202.txt'
                
# load trial
filename = os.path.join(current_dir, data_path, file)
df = pd.read_csv(filename,index_col=False,sep=r',|\t', engine='python')
df = df.replace('\t','', regex=True)
#print(df.iloc[[0]].keys())
# to numpy
posDatas_ = df.to_numpy()
posDatas = posDatas_[:, [4, 5, 6] + [13+i for i in range(94-13)]] # cut away unnecessary data
posDatas = posDatas.reshape(posDatas.shape[0], int(posDatas.shape[1]/3), 3)
posDatas = posDatas[:, [i for i in range(posDatas.shape[1]) if i not in [0, 2, 3, 8, 13, 18]]] # cut away unnecessary markers

#print(posDatas.shape)
# check if labels are correct
#f = 11
# frame = (R.from_euler('XYZ', [90, 0, 0], degrees=True).as_matrix().squeeze() @ posDatas[f].T).T
# print(frame)
# fig = go.Figure()
# plot_hand(fig, frame, names=np.arange(0, frame.shape[0]))
# fig.show()

occlusions = np.zeros([posDatas.shape[0], 5])
for f in range(posDatas.shape[0]):  #for each frame
    # rotate data such that z points up
    frame = (R.from_euler('XYZ', [90, 0, 0], degrees=True).as_matrix().squeeze() @ posDatas[f].T).T
    frame = frame.astype(np.float64)
    # transform data in global frame not necessary as we only have one LMC
    if pd.isnull(frame).any(): # the LMC did not detect the hand:
        occlusions[f, :] = np.array([-1]*5)
        continue
    
    # get data for ray tracing
    handMarkers, palmMarkers, palmPlnNormals, palmCentroids, fingers, forearm_vecs = make_hand_poses(np.array([frame]))   
    occlusions_, _ = check_LMC_Hand_visibility(LMC, pos_markers=handMarkers[0], fingers_idx=fingers_idx_LMC, finger_lines=fingers[0], 
                                                finger_radius=finger_radius, palm_plane_normal=palmPlnNormals[0], 
                                                palm_centroid=palmCentroids[0], palm_markers=palmMarkers[0], forearm_vec=forearm_vecs[0],
                                                fig = None, verbose=0)

    occlusions[f, :] = occlusions_[2:]
    #print('    Hand detected: {}'.format(f))